### Given a fic_id, find its author_key, pseudos

In [78]:
import pandas as pd
import json
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pprint
from datetime import datetime
import numpy as np
import requests
from bs4 import BeautifulSoup
import argparse
import time
import os
import csv
import sys
from unidecode import unidecode
from itertools import groupby
from operator import itemgetter
import ast

In [158]:
def find_author_of_ficid(fic_id):
    """para: fic_id(str)"""
    
    r1 = requests.get("https://archiveofourown.org/works/" + fic_id,
                        headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'})
    soup1 = BeautifulSoup(r1.content, "html.parser")
    try:
        url_splits = soup1.find("a", rel="author")['href'].split('/')
        author_key = url_splits[2]
        pseud = url_splits[-1]
        return author_key, pseud
    except:
        print('Cannot find author for', fic_id)
        return None,None

In [159]:
find_author_of_ficid("17445209")

('falmarien', 'falmarien')

### Add author_key and pseud columns to ```TSNE_embed.csv```
 Generates ```TSNE_embed_wAuthor.csv```

In [185]:
tsne_file = 'sampledata/relationship_tsne_embed.csv'

In [191]:
def find_partner_of_row(index,row):
    rels_of_char = list(filter(lambda k: row['character'] in k.lower(), row['relationship']))

    # if this row(char) has no relationship tags
    if len(rels_of_char) == 0:
        partner = None
        partner_others = None
        
    # if this row(char) has relationship tags
    elif len(rels_of_char) > 0:
        
        partner = []
        partner_others = []
        
        # iter over the relationship tags
        for rel_of_char in rels_of_char:
            
            # find the other chararcter in the relationship
            rel_of_char = rel_of_char.lower().split('/')
            the_other = [char for char in rel_of_char if row['character'] not in char][0]
            
            if the_other in grouped[row['fic_id']]:
                # find the relationship can be shown in the dataviz
                partner.append(char)
                
            else:
                for char in all_chars:
                    if char in the_other:
                        partner_others.append(char)
                    else:
                        partner_others.append('other-' + char)
                        
    else:
        print("WHAT")
                        
    return (partner, partner_others)

In [193]:
dftsne = pd.read_csv(tsne_file, index_col = False, dtype = {'fic_id': str} )

# dftsne['row_id'] = dftsne.index
dftsne['author_key'] = ''
dftsne['pseud'] = ''
dftsne['partner'] = ''
dftsne['partner_others'] = ''

dftsne.head()

,fic_id,character,relationship,additional_tags,category,d1,d2,label,author_key,pseud,partner,partner_others
0,1084344,harry,['Draco Malfoy/Harry Potter'],"['Hogwarts Eighth Year', 'Alternate Universe -...",['M/M'],12.103756,-26.289755,Draco Malfoy/Harry Potter,,,,
1,1084344,hermione,['Draco Malfoy/Harry Potter'],"['Hogwarts Eighth Year', 'Alternate Universe -...",['M/M'],-3.762389,-0.756406,Draco Malfoy/Harry Potter,,,,
2,1084344,draco,['Draco Malfoy/Harry Potter'],"['Hogwarts Eighth Year', 'Alternate Universe -...",['M/M'],-40.697304,-17.222740,Draco Malfoy/Harry Potter,,,,
3,1084344,dumbledore,['Draco Malfoy/Harry Potter'],"['Hogwarts Eighth Year', 'Alternate Universe -...",['M/M'],-45.877621,24.786779,Draco Malfoy/Harry Potter,,,,
4,1084344,snape,['Draco Malfoy/Harry Potter'],"['Hogwarts Eighth Year', 'Alternate Universe -...",['M/M'],-38.579197,9.559690,Draco Malfoy/Harry Potter,,,,


In [194]:
for index, row in dftsne.iterrows():
    author_key, pseud = find_author_of_ficid(row['fic_id'])
    dftsne.at[index,'author_key'] = author_key
    dftsne.at[index,'pseud'] = pseud
    
    part = find_partner_of_row(index,row)
    partner = part[0]
    partner_others = part[1]
    dftsne.at[index,'partner'] = partner
    dftsne.at[index,'partnerothers'] = partner_others

    
dftsne

Cannot find author for 15959783
Cannot find author for 15959783
Cannot find author for 15959783
Cannot find author for 7796623
Cannot find author for 7796623
Cannot find author for 15781191
Cannot find author for 15781191
Cannot find author for 15781191
Cannot find author for 9276209
Cannot find author for 9276209
Cannot find author for 9276209
Cannot find author for 5324441
Cannot find author for 5324441
Cannot find author for 5324441
Cannot find author for 5324441
Cannot find author for 15084674
Cannot find author for 15084674
Cannot find author for 6283444
Cannot find author for 6283444
Cannot find author for 10047209
Cannot find author for 12592028
Cannot find author for 12592028
Cannot find author for 14318574
Cannot find author for 14318574
Cannot find author for 14318574
Cannot find author for 15704982
Cannot find author for 15704982
Cannot find author for 15704982
Cannot find author for 15704982
Cannot find author for 15704982
Cannot find author for 15704982
Cannot find author 

,fic_id,character,relationship,additional_tags,category,d1,d2,label,author_key,pseud,partner,partner_others,partnerothers
0,1084344,harry,['Draco Malfoy/Harry Potter'],"['Hogwarts Eighth Year', 'Alternate Universe -...",['M/M'],12.103756,-26.289755,Draco Malfoy/Harry Potter,LissyStage,LissyStage,None,,NaN
1,1084344,hermione,['Draco Malfoy/Harry Potter'],"['Hogwarts Eighth Year', 'Alternate Universe -...",['M/M'],-3.762389,-0.756406,Draco Malfoy/Harry Potter,LissyStage,LissyStage,None,,NaN
2,1084344,draco,['Draco Malfoy/Harry Potter'],"['Hogwarts Eighth Year', 'Alternate Universe -...",['M/M'],-40.697304,-17.222740,Draco Malfoy/Harry Potter,LissyStage,LissyStage,None,,NaN
3,1084344,dumbledore,['Draco Malfoy/Harry Potter'],"['Hogwarts Eighth Year', 'Alternate Universe -...",['M/M'],-45.877621,24.786779,Draco Malfoy/Harry Potter,LissyStage,LissyStage,None,,NaN
4,1084344,snape,['Draco Malfoy/Harry Potter'],"['Hogwarts Eighth Year', 'Alternate Universe -...",['M/M'],-38.579197,9.559690,Draco Malfoy/Harry Potter,LissyStage,LissyStage,None,,NaN
5,11089872,harry,"['Hermione Granger/Ron Weasley', 'Draco Malfoy...","['POV Draco Malfoy', 'accidentally coming out'...",['M/M'],4.685986,-39.271305,Mixed-Considered,Jmej,Jmej,None,,NaN
6,11089872,ron,"['Hermione Granger/Ron Weasley', 'Draco Malfoy...","['POV Draco Malfoy', 'accidentally coming out'...",['M/M'],17.519787,23.613609,Mixed-Considered,Jmej,Jmej,None,,NaN
7,11089872,hermione,"['Hermione Granger/Ron Weasley', 'Draco Malfoy...","['POV Draco Malfoy', 'accidentally coming out'...",['M/M'],1.035340,40.741337,Mixed-Considered,Jmej,Jmej,None,,NaN
8,11089872,draco,"['Hermione Granger/Ron Weasley', 'Draco Malfoy...","['POV Draco Malfoy', 'accidentally coming out'...",['M/M'],-32.575645,-29.106026,Mixed-Considered,Jmej,Jmej,None,,NaN
9,15959783,harry,['Draco Malfoy/Harry Potter'],"['listen .', 'i wrote this while extremely sic...",['M/M'],0.372826,-42.948978,Draco Malfoy/Harry Potter,None,None,None,,NaN


In [195]:
dftsne.to_csv('sampledata/relationship_tsne_embed_wAuthorPartner.csv')

# deprecated

In [ ]:
# input tsne_embed.csv
tsne_fin = open('sampledata/relationship_tsne_embed.csv')
rows = csv.reader(tsne_fin)
row0 = next(rows)
row0.append('author_key, pseud')
print(row0)

# output tsne_embed_wAuthor.csv
fout = open(tsne_file[:-4] + '_wAuthor.csv', mode = "w+")
foutwriter = csv.writer(fout)
foutwriter.writerow(row0)

lookup = {} 

for row in rows:
    fic_id = row[0]
    
    # find author and pseudo
    # (but first, check we have looked up this fic_id already)
    if fic_id in lookup:
        author_key = lookup[fic_id][0]
        pseud = lookup[fic_id][1]
    else:
        author_key, pseud = find_author_of_ficid(fic_id)
        
    foutwriter.writerow(row + [author_key, pseud])

tsne_fin.close()
fout.close()

### Add a partner column to ```TSNE_embed_wAuthor.csv```

In [142]:
df = pd.read_csv('sampledata/relationship_tsne_embed_wAuthor.csv',
                 index_col=False,
                converters={'relationship':ast.literal_eval}) #parse 'relationship' as list

df

# find all characters in each fic.
grouped = df.groupby('fic_id')['character'].apply(list)

# find all characters in this csv
all_chars = list(set(df['character']))
all_chars

['snape',
 'harry',
 'dumbledore',
 'ginny',
 'hermione',
 'draco',
 'ron',
 'remus',
 'sirius']

In [155]:
# iterate over each row (each character in a fic)
for index, row in df.iterrows():

    rels_of_char = list(filter(lambda k: row['character'] in k.lower(), row['relationship']))
    
    # if this row(char) has no relationship tags
    if len(rels_of_char) == 0:
        partener = None
        
    # if this row(char) has relationship tags
    elif len(rels_of_char) > 0:
        
        partner = []
        partner_others = []
        
        # iter over the relationship tags
        for rel_of_char in rels_of_char:
            
            # find the other chararcter in the relationship
            rel_of_char = rel_of_char.lower().split('/')
            the_other = [char for char in rel_of_char if row['character'] not in char][0]
            
            if the_other in grouped[row['fic_id']]:
                # find the relationship can be shown in the dataviz
                partner.append(char)
                
            else:
                for char in all_chars:
                    if char in the_other:
                        partner_others.append(char)
                    else:
                        partner_others.append('other-' + char)
                
                    
    #print(row['character'], partner)
    df.at[index,'partner'] = partner
    df.at[index,'partner_others'] = partner_others

df.to_csv('sampledata/relationship_tsne_embed_wAuthorPartner.csv')
    


ValueError: Must have equal len keys and value when setting with an iterable